In [1]:
import surprise 

print(surprise.__version__)

1.1.1


### Surprise 를 이용한 추천 시스템 구축

In [2]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [3]:
# 실행하면 다음 오류가 발생합니다.
#"URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>"
# 데이터셋을 제공하는 grouplens.org 사이트에 https가 아닌 http로 요청하면 응답하지 않는 것이 원인으로 보입니다.
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\yong/.surprise_data/ml-100k


In [4]:
# 조치 방법 1-1
# 설치된 surprise 패키지의 builtin_datasets.py 파일 경로를 확인
print(surprise.__file__)

C:\Users\yong\anaconda3\envs\pymldgrev2\lib\site-packages\surprise\__init__.py


In [5]:
# 조치 방법 1-2
# BUILTIN_DATASETS 딕셔너리의 'ml-100k' 의 url 문자열의 'http'를 'https'로 수정.
!type C:\Users\yong\anaconda3\envs\pymldgrev2\lib\site-packages\surprise\builtin_datasets.py

"""This module contains built-in datasets that can be automatically
downloaded."""

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import zipfile
import os
import errno

from os.path import join
from collections import namedtuple
from six.moves.urllib.request import urlretrieve


def get_dataset_dir():
    """Return folder where downloaded datasets and other data are stored.
    Default folder is ~/.surprise_data/, but it can also be set by the
    environment variable ``SURPRISE_DATA_FOLDER``.
    """

    folder = os.environ.get('SURPRISE_DATA_FOLDER', os.path.expanduser('~') +
                            '/.surprise_data/')
    try:
        os.makedirs(folder)
    except OSError as e:
        if e.errno != errno.EEXIST:
            # reraise exception if folder does not exist and creation failed.
            raise

    return folder


# a builtin dataset has
# - an url (where to download it)
# - a path (where it is locat

In [6]:
# 조치 방법 2-1
# BUILTIN_DATASETS 딕셔너리 확인
surprise.builtin_datasets.BUILTIN_DATASETS

{'ml-100k': BuiltinDataset(url='http://files.grouplens.org/datasets/movielens/ml-100k.zip', path='C:\\Users\\yong/.surprise_data/ml-100k/ml-100k/u.data', reader_params={'line_format': 'user item rating timestamp', 'rating_scale': (1, 5), 'sep': '\t'}),
 'ml-1m': BuiltinDataset(url='http://files.grouplens.org/datasets/movielens/ml-1m.zip', path='C:\\Users\\yong/.surprise_data/ml-1m/ml-1m/ratings.dat', reader_params={'line_format': 'user item rating timestamp', 'rating_scale': (1, 5), 'sep': '::'}),
 'jester': BuiltinDataset(url='http://eigentaste.berkeley.edu/dataset/archive/jester_dataset_2.zip', path='C:\\Users\\yong/.surprise_data/jester/jester_ratings.dat', reader_params={'line_format': 'user item rating', 'rating_scale': (-10, 10)})}

In [8]:
# 조치 방법 2-2
# BUILTIN_DATASETS 딕셔너리의 'ml-100k'를 교체
surprise.builtin_datasets.BUILTIN_DATASETS['ml-100k'] = surprise.builtin_datasets.BuiltinDataset(url='https://files.grouplens.org/datasets/movielens/ml-100k.zip', path='C:\\Users\\yong/.surprise_data/ml-100k/ml-100k/u.data', reader_params={'line_format': 'user item rating timestamp', 'rating_scale': (1, 5), 'sep': '\t'})

In [9]:
# 재시도
data = Dataset.load_builtin('ml-100k') 

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\yong/.surprise_data/ml-100k


In [10]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

In [11]:
algo = SVD()
algo.fit(trainset) 

In [12]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.636089959951193, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.9020730610380165, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.996625104928444, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.7176669164848013, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3794697362295243, details={'was_impossible': False})]

In [13]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.636089959951193),
 ('882', '291', 3.9020730610380165),
 ('535', '507', 3.996625104928444)]

In [14]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.17   {'was_impossible': False}


In [15]:
accuracy.rmse(predictions)

RMSE: 0.9466


0.9465747505874536

### Surprise 주요 모듈 소개

In [16]:
import pandas as pd

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)


In [17]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

In [18]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

In [19]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

### 교차 검증과 하이퍼 파라미터 튜닝

In [23]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8777  0.8715  0.8722  0.8716  0.8706  0.8727  0.0025  
MAE (testset)     0.6734  0.6709  0.6725  0.6695  0.6672  0.6707  0.0022  
Fit time          3.15    3.18    3.16    3.12    3.15    3.15    0.02    
Test time         0.09    0.09    0.19    0.09    0.09    0.11    0.04    


{'test_rmse': array([0.87774261, 0.87151004, 0.87222609, 0.87161525, 0.87063444]),
 'test_mae': array([0.67339897, 0.67094638, 0.67250519, 0.66948402, 0.66718352]),
 'fit_time': (3.1489789485931396,
  3.181527853012085,
  3.1582796573638916,
  3.1223626136779785,
  3.1519360542297363),
 'test_time': (0.09416532516479492,
  0.09424495697021484,
  0.1884136199951172,
  0.09424066543579102,
  0.09425878524780273)}

In [24]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8767973513293925
{'n_epochs': 20, 'n_factors': 50}


### Surprise 를 이용한 개인화 영화 추천 시스템 구축

In [25]:
# 다음 코드는 train_test_split( )으로 분리되지 않는 데이터 세트에 fit( )을 호출해 오류가 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)


AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [26]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()


In [27]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)


In [28]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])


사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [29]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)


user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [30]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [31]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
